In [7]:
pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [17]:
pip install feedparser

     ---------------------------------------- 81.1/81.1 KB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
  Running setup.py install for sgmllib3k: started
  Running setup.py install for sgmllib3k: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib.robotparser
import xmltodict
import requests
import feedparser
import os
import requests
import re

### 로이터 뉴스 

### URL 생성하기

In [2]:
# robots.txt해석
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.reuters.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.reuters.com/sitemap.xml')

True

In [3]:
rp.can_fetch('*', 'https:/www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml')

True

In [4]:
rp.can_fetch("*", "https://www.reuters.com/finance/stocks/option")

False

#### sitemap.xml 파일로 URL 획득

In [5]:
sitemap = xmltodict.parse(requests.get(
    'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml').text
)

In [6]:
# 파일을 다운로드 하기 전에 dict에 무엇이 있는지 확인해보기
urls = [url['loc'] for url in sitemap['urlset']['url']]
# 공간을 차지하지 않도록 맨 앞 URL 몇 개만 출력한다.
print("\n".join(urls[0:3]))

https://www.reuters.com/sports/baseball/ryne-nelson-diamondbacks-handle-angels-2023-07-02/
https://www.reuters.com/sports/soccer/former-man-utd-midfielder-cleverley-announces-retirement-2023-07-02/
https://www.reuters.com/world/americas/guatemala-court-orders-presidential-ballot-review-opposition-cries-foul-2023-07-02/


#### RSS에서 URL 획득

In [7]:
import feedparser
feed = feedparser.parse('http://web.archive.org/web/20200613003232if_/http://feeds.reuters.com/Reuters/worldNews')

In [8]:
[(e.title, e.link) for e in feed.entries]

[('Mexico City to begin gradual exit from lockdown on Monday',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/OQtkVdAqHos/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R'),
 ('Mexico reports record tally of 5,222 new coronavirus cases',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/Rkz9j2G7lJU/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B'),
 ('Venezuela supreme court to swear in new electoral council leaders, government says',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/cc3R5aq4Ksk/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T'),
 ("One-fifth of Britain's coronavirus patients were infected in hospitals: Telegraph",
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/1_7Wb0S_6-8/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382'),
 ('France to lift border controls for EU travellers on June 15',
  'http://feeds.reuters.com/~r/

In [9]:
# 실제 URL 뽑아보기
[e.id for e in feed.entries]

['https://www.reuters.com/article/us-health-coronavirus-mexico-city/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-mexico/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-venezuela-politics/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-britain-hospitals/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-france-borders/france-to-lift-border-controls-for-eu-travellers-on-june-15-idUSKBN23J385?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-brazil/brazils-covid-19-deaths-sur

### 데이터 다운로드

### 파이썬을 사용한 HTML 페이지 다운로드

In [11]:
%%time
output_directory = './data/'
s = requests.Session()
for url in urls[0:10] : 
    # URL의 마지막/뒤 부분을 가져와서 파일 이름으로 사용한다.
    file = url.split("/")[-1]
    # file_path = os.path.join(output_directory, file)
    
    r = s.get(url)
    if r.ok : 
        with open('./file', 'w+b') as f : 
            f.write(r.text.encode('utf-8'))
    else : 
        print('error with URL %s' % url)

CPU times: total: 15.6 ms
Wall time: 4.95 s


#### wget를 사용한 HTML 페이지 다운로드

In [12]:
with open('urls.txt', 'w+b') as f : 
    f.write('\n'.join(urls).encode('utf-8'))

### 반정형 데이터 추출

#### 정규 표현식을 사용한 추출

In [15]:
url = 'https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

# 마지막 / 뒤의 부분을 파일 이름으로 사용하기
file = url.split('/')[-1] + '.html'
r = requests.get(url)
with open(file, 'w+b') as f : 
    f.write(r.text.encode('utf-8'))

In [22]:
with open(file, "r", encoding='utf-8') as f:
    html = f.read()
    g = re.search(r'<title>(.*)</title>', html, re.MULTILINE|re.DOTALL)
    if g:
        print(g.groups()[0])

Banned in Boston: Without vaping, medical marijuana patients must adapt | Reuters
